# Setup

## Import TensorFlow and NumPy

In [1]:
# Import libraries
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.datasets import cifar10

## Configure DNN settings

Here, we specify the ResNet architecture parameters:

In [2]:
# Number of classes to infer
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Depth parameter
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

## Load dataset and preprocess

We are working with the [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html) here.

In [3]:
# Load the CIFAR10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions
input_shape = x_train.shape[1:]

# Normalize data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


## Load trained ResNet model

In [4]:
# Load model
model_name = 'weight_quant_4b_final_3'

# Model name, depth, and version
model_type = 'ResNet%dv%d_%s' % (depth, version, model_name)
print(model_type)

# Prepare model saving directory
save_dir = os.path.join(os.getcwd(), model_type)
model_full_name = 'cifar10_%s_model' % model_type
filepath = os.path.join(save_dir, model_full_name)

# Load model checkpoint
K.clear_session()
model = load_model(filepath)

ResNet20v1_weight_quant_4b_final_3


In [5]:
# Save all model parameters to a dict
weights = {}
for olayer in model.layers:
    weights[olayer.name] = olayer.get_weights()

In [6]:
# Perform baseline inference before quantization
model.evaluate(x_test, y_test, verbose=1)

313/313 [==============================] - 20s 61ms/step - loss: 0.3958 - accuracy: 0.9030


[0.39580658078193665, 0.902999997138977]

In [7]:
# Perform quantized inference
weights_quant = {}
weights_enci = {}
for layer_name in weights:
    if 'conv2d' in layer_name or 'dense' in layer_name:
        # Get weights from layer
        print(layer_name)
        W, b, W_max = weights[layer_name]

        # Get quantized weights
        weights_quant[layer_name] = tf.quantization.fake_quant_with_min_max_args(W, -W_max, W_max, 4, narrow_range=True).numpy()
        
        # Get quantized weights encoding index
        weights_enci[layer_name] = np.int8(np.round(weights_quant[layer_name] / W_max * 7))
        positivize = lambda x : x + 1 if x > 0 else x
        weights_enci[layer_name] = np.vectorize(positivize)(weights_enci[layer_name]) + 7
        
        print(sorted(np.unique(weights_enci[layer_name])))


conv2d_noise
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_1
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_2
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_3
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_4
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_5
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_6
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_7
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_8
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_9
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_10
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_11
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_12
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_13
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15]
conv2d_noise_14
[0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 

In [8]:
# Create weight matrices after RRAM relaxation according to the confusion matrices (4 levels per cell)
for seed in range(10):
    # Set random seed for statistics
    np.random.seed(seed)

    # Sweep indices
    for i in range(16):
        for j in range(16):
            # Ignore diagonal entries
            if i == j:
                continue

            # Sweep BERs
            for ber in 10.**np.linspace(-8, 0, 17):
                # Log
                print(f'Doing {seed}\t{i}\t{j}\t{ber}')

                # Create confusion matrix
                C = np.eye(16)
                C[i][i] = 1-ber
                C[i][j] = ber

                # Perturb weights under confmat error model
                weights_perturb = {}
                for layer_name in weights:
                    if 'conv2d' in layer_name or 'dense' in layer_name:
                        W, b, W_max = weights[layer_name]

                        # Perturb based on confusion matrix
                        perturb = lambda x : np.random.choice(16, p=C[x])
                        weights_perturb[layer_name] = np.vectorize(perturb)(weights_enci[layer_name])
                        print("BLAH1")
                        print(weights_enci[layer_name])
                        print("BLAH1.5")
                        print(weights_perturb[layer_name])
                        print("BLAH2")

                        # Scale back to weight value
                        depositivize = lambda x : x - 1 if x > 7 else x
                        weights_perturb[layer_name] = (np.vectorize(depositivize)(weights_perturb[layer_name]) - 7) * W_max / 7

                # Load relaxed weights back to the model
                for layer in model.layers:
                    if layer.name in weights_perturb:
                        print(layer.name)
                        W, b, W_max = layer.get_weights()
                        layer.set_weights([weights_perturb[layer.name], b, W_max])

                # Evaluate accuracy after relaxation
                loss, accuracy = model.evaluate(x_test, y_test, verbose=1)

                # Append results to file
                with open('char.tsv', 'a') as outf:
                    outf.write(f'{seed}\t{i}\t{j}\t{ber}\t{loss}\t{accuracy}\n')


Doing 0	0	1	1e-08
BLAH1
[[[[ 4 10  9  4 11  9  1  4  3  3  6  3  7  7  4  9]
   [12 11  7 10  3  5  4  3  7  6  9  7 11  7  5  6]
   [12  7 12  7  6 11 10  2  7  9 12  5  2  6  7  7]]

  [[10  9  6  7 12  9  2  6  7 10  5  4 15  3 12 10]
   [ 9  4  9 11  2 11  6  4 10  3  5 10  7  4  5 10]
   [ 9  5 13 13  3  4 10  3 12  7 10  7  3  4 11  5]]

  [[ 7  7  7  7 10  4 10  4  3  4  9  9 11 11 11 12]
   [ 9  5  7  7  9  9 11  6  7  5  7 11  4 10  4  7]
   [ 7  5  9  6  6  1 10  7  9  7 12  5  7 11  7  9]]]


 [[[ 3 13  7  5 11  4 11  7  9 13  2  4 11  2  5  7]
   [10 13 10  5  2 10 12  7  6 14  9 13 12  2 10 11]
   [10 12 12  7  3 13 14  9  7 10 11  3 10  2 11  9]]

  [[13  7  7  6 14  7  0 11 15  9  3  7 11  0 11  1]
   [11 10  6  0  4 12  5  9 14  0  3 15 10  1  4  6]
   [ 9 11 14  3  6  7  9 13 11  3  9  3  5  2  9  9]]

  [[14  2  3 15 10 11  6 10  9 12  6  6  7  9 10  3]
   [ 5  3  3  7  6 14  6  5  7  7  6 12  3  7  0  4]
   [ 3  4  7  9  6  4  7 11  7  9 13  3  5  9  4  2]]]


 [[[ 3

KeyboardInterrupt: 

TODO: get weight freqs